In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
#Gradio is a Python Library that helps us quickly create interactive User Interfaces (UIs) without needing to build a
#frontend from scratch.

In [2]:
import os
load_dotenv(override=True)
deepseek_key=os.getenv('DEEPSEEK_API_KEY')
if deepseek_key:
    print(f"DeepSeek Key exists and begins with :{deepseek_key}")
else:
    print("Key not found")


DeepSeek Key exists and begins with :sk-e73f32db92504de39639231a3f71f588


In [18]:
load_dotenv(override=True)
client=OpenAI()

In [17]:
deepseek=OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com/v1")

In [4]:
reader=PdfReader("me/Neeraj Resume.pdf")
resume=""
for page in reader.pages:
    text= page.extract_text()
    if text:
        resume += text


In [5]:
print(resume)

Neeraj Sandhu
+91 8091220037 |sandhuneeraj.works@gmail.com |LinkedIn: neerajsandhu
Objective
Aspiring frontend developer proficient in HTML, CSS, and JavaScript, passionate about transforming designs
into responsive, user-friendly interfaces. Excited to apply my technical skills and creative problem-solving to
build engaging web experiences in a collaborative environment
Education
Thapar Institute of Engineering and Technology Patiala, Punjab
B.E. Computer Science Engineering 2021 – 2025
St. D. R. Public Sr. Sec. School Una, Himachal Pradesh
Class 12th 2019 – 2020
Parmar International School Una, Himachal Pradesh
Class 10th 2017 – 2018
Projects
VividVue Living |HTML, CSS, JavaScript, Selenium, TestNg
•Developed a user-friendly real estate website with dynamic property listings, advanced search filters, and an
intuitive interface. Used Selenium and TestNG for automation testing based on the design of test cases to ensure
the reliability and proper functionality of the website.
Ecommerce

In [6]:
with open("me/summary.txt", "r", encoding='utf-8') as f:
    summary=f.read()

In [7]:
name = "Neeraj Sandhu"
system_prompt= f"You are acting as {name}. You are answering questions on {name}'s Website, \
    particularly questions related to {name}'s carrer, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know answer , say so."

system_prompt+=f"\n\n ## Summary:\n{summary}\n\n##{resume}\n\n"
system_prompt+=f"with this context, please chat with the user, always staying in character as {name}"

In [26]:
def chat(message,history):
    messages=[{'role': 'user', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [27]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [29]:
#Pydantic is used to define, validate, and parse data models in Python, making sure your data is clean , structured and type-safe.
#BaseModel is a core class which is used to create our own data models/schemas.
#used of pydantic :
#1. automatically checks if input data is of the right type.
#2. converts data into correct Python types.
#3. tells us exactly what's wrong if data doesn't match the schema.
#4. can read environment variables/config files and validate them automatically.
from pydantic import BaseModel

#define a schema 
class Evaluation (BaseModel):
    is_acceptable: bool
    feedback: str

In [30]:
evaluator_system_prompt= f"You are an evaluator that decides where a response to a question is acceptable. \
    You are provided with a conversation between a user and an agent. your task is to decide whether the agent's latest response is acceptable or not. \
    The agent is playing the role of {name} and is representing {name} on their website. \
    The agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who come across the website. \
    The agent has been provided with context on {name} in the form of their summary and Resume details. Here is the Information: "

evaluator_system_prompt+=f"\n\n ##Summary: \n{summary}\n\n##Resume details:\n{resume}\n\n"
evaluator_system_prompt+=f"with this context, please evaluate the latest response, replying with whether the response is acceptable or not then recommend the improved answer."

In [43]:
def evaluater_user_prompt(reply, message, history):
    user_prompt=f"Here is the conversation between a user and an agent: \n\n{history}\n\n"
    user_prompt+=f"The agent's latest response is: \n\n{reply}\n\n"
    user_prompt+=f"The user's latest question is: \n\n{message}\n\n"
    user_prompt+=f"Please evaluate the response and recommend the improved answer."
    return user_prompt

In [32]:
import os
gemini=OpenAI(
    api_key=os.getenv('GEMINI_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    messages=[{'role':'user', 'content': evaluator_system_prompt}] + [{"role":"user", "content":evaluater_user_prompt(reply, message, history)}]
    response=gemini.beta.chat.completions.parse(
        model='gemini-2.5-pro',
        messages=messages,
        response_format=Evaluation
    )
    return response.choices[0].message.parsed


In [39]:
messages=[{"role": "user", "content": system_prompt}] + [{"role": "user", "content": "do you have any projects?"}]
response=client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages
)

reply= response.choices[0].message.content


In [40]:
reply

'Yes — I’ve built several projects across frontend, backend, automation testing and ML. Here are the highlights:\n\n- VividVue Living (Real estate) — HTML, CSS, JavaScript, Selenium, TestNG  \n  I built a responsive real-estate site with dynamic property listings, advanced search filters and an intuitive UI. I also wrote Selenium + TestNG automation tests to validate core workflows and ensure reliability.\n\n- Ecommerce Website — HTML, CSS, PHP, SQL  \n  A full e‑commerce platform with user authentication, product/inventory management, order processing and a robust SQL backend. Focused on responsive design and secure data handling.\n\n- Diabetes Prediction Website — Python, ML  \n  A web app that predicts diabetes risk using ML models. I handled data preprocessing, feature selection, model training and integrated results into a user-friendly interface with visualizations to explain predictions.\n\n- Movie Recommendation System — Python, NLP  \n  A recommendation engine using NLP and co

In [44]:
evaluate(reply, "do you have any projects?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is excellent. It directly answers the question, accurately details the projects from the resume, maintains a professional and engaging tone, and concludes with a strong call to action that encourages further conversation. No improvement is necessary, as this is a model answer.')

In [45]:
def rerun(reply,message,history,feedback):
    updated_system_prompt=system_prompt + f"\n\n ##Previous answer rejected\n you just ried to reply, but the quality control rejejected your response"
    updated_system_prompt+= f"## Your attempted answer: \n{reply}\n\n"
    updated_system_prompt+= f"## Reason for rejection: \n{feedback}\n\n"
    messages = [{"role" : "System", "content" : updated_system_prompt}] + history + [{"role" : "user", "content" : message}]
    response=client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages
    )

    return response.choices[0].message.content

In [46]:
def chat(message, history):
    system= system_prompt
    messages=[{"role": "system", "content" : system}] + history + [{"role": "user", "content" : message}]
    response=client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages
    )

    reply=response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passes evaluation -  returning reply")
    else:
        print("Failed Evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply


In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Passes evaluation -  returning reply
Passes evaluation -  returning reply
